In [1]:
#default_exp train

In [2]:
#export
from rsna_retro.imports import *
from rsna_retro.metadata import *
from rsna_retro.preprocess import *

Loading imports


In [3]:
torch.cuda.set_device(4)

In [4]:
#export
def get_pil_fn(p):
    def _f(fn): return PILCTScan.create(p/f'{fn}.jpg')
    return _f

In [5]:
#export
def fn2label(fn): return Meta.df_comb.loc[fn][htypes].values.astype(np.float32)

In [6]:
#export
def get_wgts(df, splits):
    wgts = df['any'][splits[0]].values
    return wgts * (1/0.14 - 2) + 1

In [7]:
get_wgts(Meta.df_comb, Meta.splits)

array([1., 1., 1., ..., 1., 1., 1.])

In [8]:
#export
def get_data_gen(fns, bs, img_tfm, splits, sz=None, nw=8, mean=mean, std=std,
        wgts=None, batch_xtra=None, after_item=None, with_aug=True, test=False, **kwargs):
    tfms = [[img_tfm, ToTensor], [fn2label,EncodedMultiCategorize(htypes)]]
    if test: tfms = [tfms[0]]
    dsrc = DataSource(fns, tfms, splits=splits)
    nrm = Normalize.from_stats(mean,std)
    batch_tfms = L(IntToFloatTensor, nrm, Cuda()) + L(batch_xtra)
    if with_aug: batch_tfms += aug_transforms(**kwargs)
    if sz is not None:
        batch_tfms = batch_tfms+[RandomResizedCropGPU(sz, min_scale=0.7, ratio=(1.,1.), valid_scale=0.9)]
    if wgts is None:
        return dsrc.databunch(bs=bs, num_workers=nw, after_item=after_item, after_batch=batch_tfms)
    else:
        return dsrc.weighted_databunch(wgts, bs=bs, num_workers=nw, after_item=after_item, after_batch=batch_tfms)


In [9]:
#export
def filename(o): return os.path.splitext(os.path.basename(o))[0]

In [10]:
Meta.df_comb

,Modality,PatientID,StudyInstanceUID,SeriesInstanceUID,StudyID,ImagePositionPatient,ImageOrientationPatient,SamplesPerPixel,PhotometricInterpretation,Rows,...,MultiWindowCenter,WindowCenter1,MultiWindowWidth,WindowWidth1,any,epidural,intraparenchymal,intraventricular,subarachnoid,subdural
SOPInstanceUID,,,,,,,,,,,,,,,,,,,,,
ID_352e89f1c,CT,ID_d557ddd2,ID_05074a0d95,ID_be6165332c,,-125.000000,1.0,1,MONOCHROME2,512,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0
ID_3cf4fb50f,CT,ID_16b2ad86,ID_c3a404ea2e,ID_2c1454e208,,-125.000000,1.0,1,MONOCHROME2,512,...,1.0,36.0,1.0,80.0,0,0,0,0,0,0
ID_e3674b189,CT,ID_eb712bf0,ID_db83193795,ID_e1facea145,,-125.000000,1.0,1,MONOCHROME2,512,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0
ID_2a8702d25,CT,ID_ff137633,ID_d17053848c,ID_7098f7c836,,-126.437378,1.0,1,MONOCHROME2,512,...,NaN,NaN,NaN,NaN,1,0,1,1,0,0
ID_7be0f1b3c,CT,ID_cd9169c2,ID_b42de79024,ID_f5bd86b25b,,-125.000000,1.0,1,MONOCHROME2,512,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ID_6097e9f6f,CT,ID_17214181,ID_bddb5e0824,ID_fd40582b03,,-107.000000,1.0,1,MONOCHROME2,512,...,1.0,36.0,1.0,80.0,0,0,0,0,0,0
ID_d0353cdfe,CT,ID_be09d60c,ID_dd98962b72,ID_3038ce1582,,-125.000000,1.0,1,MONOCHROME2,512,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0
ID_60186961d,CT,ID_fe4dc149,ID_08eaca3179,ID_f522d564e7,,-117.500000,1.0,1,MONOCHROME2,512,...,1.0,36.0,1.0,80.0,0,0,0,0,0,0


In [11]:
#export
def get_data(bs, sz, splits, img_dir=path_jpg256):
    return get_data_gen(L(list(Meta.df_comb.index)), bs=bs, img_tfm=get_pil_fn(path/img_dir), 
                        sz=sz, splits=splits)

In [12]:
%time get_data(512, 128, Meta.splits_sample)

CPU times: user 3.77 s, sys: 1.45 s, total: 5.22 s
Wall time: 5.97 s


In [13]:
%time dbch = get_data(512, 128, Meta.splits)

CPU times: user 2.47 s, sys: 43.6 ms, total: 2.51 s
Wall time: 2.51 s


In [14]:
get_data(512, 128, Meta.splits_sample)

## Training

In [19]:
#export
def accuracy_any(inp, targ, thresh=0.5, sigmoid=True):
    inp,targ = flatten_check(inp[:,0],targ[:,0])
    if sigmoid: inp = inp.sigmoid()
    return ((inp>thresh)==targ.bool()).float().mean()


def get_loss(scale=None):
    num_classes = 6
    loss_weights = to_device(tensor(2.0, 1, 1, 1, 1, 1))
    loss_weights = loss_weights/loss_weights.sum()*num_classes
    
    if scale is not None: scale = to_device(tensor([scale]*num_classes))
    return BaseLoss(nn.BCEWithLogitsLoss, weight=loss_weights, #pos_weight=scale,
                    floatify=True, flatten=False, is_2d=False, activation=torch.sigmoid)


In [20]:
#export
def get_learner(dbch, arch_or_model, lf=None, pretrained=False, opt_func=None, metrics=None, fp16=True, config=None):
    if lf is None: lf = get_loss()
    if metrics is None: metrics=[accuracy_multi,accuracy_any]
    if opt_func is None: opt_func = partial(Adam, wd=1e-5, eps=1e-4, sqr_mom=0.999)
    if isinstance(arch_or_model, nn.Module):
        learn = Learner(dbch, arch_or_model, loss_func=lf, lr=3e-3,
                    opt_func=opt_func, metrics=metrics)
    else:
        if config is None: config=dict(ps=0., lin_ftrs=[], concat_pool=False)
        learn = cnn_learner(dbch, arch_or_model, pretrained=pretrained, loss_func=lf, lr=3e-3,
                            opt_func=opt_func, metrics=metrics, config=config)
    return learn.to_fp16() if fp16 else learn

In [21]:
# def fit_tune(bs, sz, epochs, lr):
#     learn.dbunch = get_data(bs, sz)
#     do_fit(learn, epochs, lr, freeze=False)

In [22]:
#export
moms=(0.9,0.9,0.9)
no_1cycle = dict(div=1, div_final=1, pct_start=0.1, moms=moms)

def do_fit(learn, epochs, lr, freeze=False, do_slice=False, **kwargs):
    if do_slice: lr = slice(lr*3,lr)
    if freeze:
        learn.freeze()
        learn.fit_one_cycle(1, lr, div=2, div_final=1, pct_start=0.1)
    learn.unfreeze()
    learn.fit_one_cycle(epochs, lr, **kwargs)

## Submission

In [26]:
#export
def get_test_data(df_tst, bs=512, sz=256, tst_dir='tst_jpg', sl=None):
    tst_fns = df_tst.index.values
    if sl is not None: tst_fns = tst_fns[sl]
    tst_splits = [L.range(tst_fns), L.range(tst_fns)]
    tst_dbch = get_data_gen(tst_fns, bs=bs, img_tfm=get_pil_fn(path/tst_dir), sz=sz, splits=tst_splits, test=True)
    tst_dbch.c = 6
    return tst_dbch

In [27]:
#export
def submission(df_tst, preds, fn='submission'):
    ids,labels = [],[]
    for idx,pred in zip(df_tst.index, preds):
        for i,label in enumerate(htypes):
            ids.append(f"{idx}_{label}")
            labels.append('{0:1.10f}'.format(pred[i].item()))
    df_csv = pd.DataFrame({'ID': ids, 'Label': labels})
    df_csv.to_csv(f'{fn}.csv', index=False)
    return df_csv

## Example

In [28]:
name = 'sample'

In [29]:
dbch = get_data(512, 128, splits=Meta.splits_sample)
learn = get_learner(dbch, xresnet18)

In [30]:
do_fit(learn, 1, 1e-2)
learn.save(f'runs/{name}-1')

epoch,train_loss,valid_loss,accuracy_multi,accuracy_any,time
0,0.710802,0.435682,0.830323,0.769416,00:18


In [19]:
learn.load(f'runs/{name}-1')

## Example Submission

In [27]:
learn.dbunch = get_test_data(Meta.df_tst, bs=256, sz=256)

In [28]:
# tst = test_dl(learn.dbunch, tst_fns, after_item=[get_pil_fn(path/'tst_jpg'), ToTensor])

# tst.tfms = Pipeline(funcs=[get_pil_fn(path/'tst_jpg'), ToTensor])

In [29]:
sub_fn = f'subm/{name}'

In [ ]:
preds,targs = learn.get_preds()

In [33]:
pred_csv = submission(df_tst, preds, fn=sub_fn)

In [ ]:
FileLink(f'{sub_fn}.csv')

In [7]:
api.competition_submit(f'{sub_fn}.csv', 'testing rsna_retro submission', 'rsna-intracranial-hemorrhage-detection')

100%|██████████| 26.0M/26.0M [00:03<00:00, 8.99MB/s]


Successfully submitted to RSNA Intracranial Hemorrhage Detection

In [15]:
api.competitions_submissions_list('rsna-intracranial-hemorrhage-detection')[0]

{'ref': 13695014,
 'totalBytes': 27277209,
 'date': '2019-12-08T22:49:07.013Z',
 'description': 'testing rsna_retro submission',
 'errorDescription': None,
 'fileName': 'test_sub.csv',
 'publicScore': '0.95475',
 'privateScore': '0.06444',
 'status': 'complete',
 'submittedBy': 'Andrew Shaw',
 'submittedByRef': 'bearpelican',
 'teamName': 'Andrew Shaw',
 'type': 'standard',
 'url': 'https://www.kaggle.com/submissions/13695014/13695014.raw'}

## Save Predictions

In [21]:
#export
class DummyLoss:
    def __call__(self, p, *t, **kwargs): return torch.tensor(0, device=p.device).float()

In [22]:
#export
def save_features(learn, feat_path):
    preds,targs = learn.get_preds(dl=learn.dbunch.valid_dl)
    val_ids = dbunch.valid_dl.dataset.items
    feat_path.mkdir(exist_ok=True)
    for idx,pred in progress_bar(zip(val_ids, preds), total=len(val_ids)):
        np.save(str(feat_path/f'{idx}'), pred.squeeze().numpy())

In [23]:
fn_save = 'runs/baseline_any_xse_resnext_512_appian-1'

In [24]:
#export
path_feat256 = path/'features_512'
path_feat256_tst = path/'tst_features_512'

In [ ]:
dbunch = get_test_data(Meta.df_comb, bs=128, sz=None, tst_dir='nocrop_jpg')
learn = get_learner(dbunch, xresnet34, lf=DummyLoss(), metrics=[])
learn.load(fn_save)
learn.model = learn.model[0]

In [23]:
save_features(learn, path_feat256)

In [24]:
dbunch = get_test_data(Meta.df_tst, bs=512, sz=None, tst_dir='tst_jpg')
learn = get_learner(dbunch, xresnet34, lf=DummyLoss(), metrics=[])
learn.load(fn_save)
learn.model = learn.model[0]
save_features(learn, path_feat256_tst)

## Export

In [2]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_metadata.ipynb.
Converted 01_preprocess.ipynb.
Converted 01_preprocess_mean_std.ipynb.
Converted 02_train.ipynb.
Converted 03_train3d.ipynb.
Converted 03_train3d_01_train3d.ipynb.
Converted 03_train3d_01b_train_lstm.ipynb.
Converted 03_train3d_02_train_head.ipynb.
Converted 03_trainfull3d.ipynb.
Converted 04_trainSeq_01_lstm.ipynb.
Converted 04_trainSeq_02_transformer.ipynb.
Converted 04_trainSeq_03_lstm_seutao.ipynb.
Converted 05_train_adjacent.ipynb.
Converted 05_train_adjacent_01_5c.ipynb.
Converted 05_train_adjacent_02_3c.ipynb.
Converted 06_seutao_features.ipynb.
Converted 06_seutao_features_01_simple_lstm_20ep.ipynb.
Converted 06_seutao_features_01b_simple_lstm_10ep.ipynb.
Converted 06_seutao_features_01c_simple_lstm_meta.ipynb.
Converted 06_seutao_features_01d_simple_lstm_meta_fulldataset.ipynb.
Converted 06_seutao_features_02_2ndPlace.ipynb.
Converted 06_seutao_features_03_1stPlace.ipynb.
Converted 07_train_3d_lstm.ipynb.
